In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
park = pd.read_csv('park.csv')
daycare = pd.read_csv('day_care_center.csv')
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [3]:
train_df.shape, test_df.shape, daycare.shape, park.shape

((1216553, 13), (5463, 12), (7551, 11), (1359, 13))

# 1. 데이터 병합하기

## 1) train에 gu 정보 추가하기 
- park의 구, 동 정보 및 서울특별시, 부산광역시 공공데이터를 추가로 이용해서 train에 구 정보 추가

In [4]:
#park의 구 정보를 동을 기준으로 train에 병합
train_df = pd.merge(train_df, park[['gu', 'dong']], on='dong', how='left')
train_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,gu
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500,NaN
1,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.17,1973,200801,1~10,6,20000,NaN
2,2,3578,서울특별시,필운동,174-1,두레엘리시안,필운동 174-1 두레엘리시안,84.74,2007,200801,1~10,6,38500,NaN
3,3,10957,서울특별시,내수동,95,파크팰리스,내수동 95 파크팰리스,146.39,2003,200801,11~20,15,118000,NaN
4,4,10639,서울특별시,내수동,110-15,킹스매너,내수동 110-15 킹스매너,194.43,2004,200801,21~31,3,120000,NaN


In [5]:
#비어있는 값 확인, 공공데이터로 나머지를 채워주도록 한다
train_df.isna().sum()

transaction_id                 0
apartment_id                   0
city                           0
dong                           0
jibun                          0
apt                            0
addr_kr                        0
exclusive_use_area             0
year_of_completion             0
transaction_year_month         0
transaction_date               0
floor                          0
transaction_real_price         0
gu                        277309
dtype: int64

In [6]:
#가장 최근 데이터를 확인해보니 2017년이다. 2018년 기준으로 행정구역을 가져오도록 한다.
train_df['transaction_year_month'].max()

201711

* 부산시 행정구역 : https://data.busan.go.kr/dataSet/detail.nm?publicdatapk=OA_CD00003&contentId=10 (https://kosis.kr/statHtml/statHtml.do?orgId=202&tblId=DT_206&lang_mode=ko&vw_cd=MT_OTITLE&list_id=202_20203_B&conn_path=I4)
* 서울시 행정구역 : https://data.seoul.go.kr/dataList/10043/S/2/datasetView.do

In [7]:
#부산시 행정구역 추가하기
busan = pd.read_csv('부산시행정구역(동별).csv', encoding='euc-kr')
busan.rename(columns={'구·군별(1)':'gu', '구·군별(2)':'dong'}, inplace=True)
busan.drop([0,1,2], inplace=True)
busan = busan[busan['dong']!= '소계']
busan = busan[['gu','dong']]
busan['city'] = '부산광역시'
busan

,gu,dong,city
4,중구,중앙동,부산광역시
5,중구,동광동,부산광역시
6,중구,대청동,부산광역시
7,중구,보수동,부산광역시
8,중구,부평동,부산광역시
...,...,...,...
221,기장군,기장읍,부산광역시
222,기장군,장안읍,부산광역시
223,기장군,정관읍,부산광역시
224,기장군,일광읍,부산광역시


In [15]:
#서울시 행정구역 추가하기
seoul = pd.read_csv('서울시행정구역(동별).csv')
seoul.rename(columns={'동별(2)':'gu', '동별(3)':'dong'}, inplace=True)
seoul.drop([0,1], inplace=True)
seoul = seoul[seoul['dong']!= '소계']
seoul['city'] = '서울특별시'
seoul = seoul[['city','gu','dong']]
seoul

,city,gu,dong
3,서울특별시,종로구,사직동
4,서울특별시,종로구,삼청동
5,서울특별시,종로구,부암동
6,서울특별시,종로구,평창동
7,서울특별시,종로구,무악동
...,...,...,...
446,서울특별시,강동구,성내3동
447,서울특별시,강동구,둔촌1동
448,서울특별시,강동구,둔촌2동
449,서울특별시,강동구,길동


In [20]:
#gu가 공란인 데이터만 처리해줄거니까 따로 빼준다
missing_gu = train_df[train_df['gu'].isna()]

In [21]:
#seoul 데이터셋의 해당 동, 구 정보를 가져와서 train의 gu열 채우기
for index, row in missing_gu.iterrows():
    if row['city'] =='서울특별시':
        gu_value = seoul[(seoul['dong'] == row['dong'])]['gu'].values
        if len(gu_value) >0:
            train_df.at[index, 'gu'] = gu_value[0]

In [27]:
#busan 데이터셋의 해당 동, 구 정보를 가져와서 train의 gu열 채우기           
for index, row in missing_gu.iterrows():
    if row['city'] == '부산광역시':
        gu_value = busan[(busan['dong'] == row['dong'])]['gu'].values
        if len(gu_value) > 0:
            train_df.at[index, 'gu'] = gu_value[0]

In [28]:
train_df.isna().sum()

transaction_id                0
apartment_id                  0
city                          0
dong                          0
jibun                         0
apt                           0
addr_kr                       0
exclusive_use_area            0
year_of_completion            0
transaction_year_month        0
transaction_date              0
floor                         0
transaction_real_price        0
gu                        51825
dtype: int64

- 동별 주민등록인구통계는 행정동 경계를 기준으로 작성하고 있다
- 예) 누락된 <성북구 상월곡동> 등은 법정동으로, 행정동으로는 월곡2동에 포함되어 있다
- 모든 동단위 통계는 행정구역(행정동) 단위로 작성하고 있으며, 법정동 단위 자료는 따로 생산되지 않아 자료를 구하는 데 한계가 있다

In [23]:
gu = pd.read_csv('전국행정동리스트.csv')
gu.rename(columns={'전국행정동리스트':'city', 'Unnamed: 2':'gu', 'Unnamed: 3':'dong'}, inplace=True)
gu.drop([0,1], inplace=True)
gu = gu[['city','gu','dong']]
gu[(gu['city']=='서울특별시') | (gu['city']=='부산광역시')]

,city,gu,dong
2,서울특별시,종로구,청운동
3,서울특별시,종로구,신교동
4,서울특별시,종로구,궁정동
5,서울특별시,종로구,효자동
6,서울특별시,종로구,창성동
...,...,...,...
558,부산광역시,NaN,구칠리
559,부산광역시,NaN,이곡리
560,부산광역시,NaN,고촌리
561,부산광역시,NaN,안평리


In [29]:
missing_gu = train_df[train_df['gu'].isna()]
                      
#seoul 데이터셋의 해당 동, 구 정보를 가져와서 train의 gu열 채우기
for index, row in missing_gu.iterrows():
    if row['city'] =='서울특별시':
        gu_value = gu[(gu['dong'] == row['dong'])]['gu'].values
        if len(gu_value) >0:
            train_df.at[index, 'gu'] = gu_value[0]
            
#busan 데이터셋의 해당 동, 구 정보를 가져와서 train의 gu열 채우기           
for index, row in missing_gu.iterrows():
    if row['city'] == '부산광역시':
        gu_value = gu[(gu['dong'] == row['dong'])]['gu'].values
        if len(gu_value) > 0:
            train_df.at[index, 'gu'] = gu_value[0]

In [30]:
train_df.isna().sum()

transaction_id                0
apartment_id                  0
city                          0
dong                          0
jibun                         0
apt                           0
addr_kr                       0
exclusive_use_area            0
year_of_completion            0
transaction_year_month        0
transaction_date              0
floor                         0
transaction_real_price        0
gu                        51825
dtype: int64

In [34]:
train_df.dropna(inplace=True)
train_df.isna().sum()

transaction_id            0
apartment_id              0
city                      0
dong                      0
jibun                     0
apt                       0
addr_kr                   0
exclusive_use_area        0
year_of_completion        0
transaction_year_month    0
transaction_date          0
floor                     0
transaction_real_price    0
gu                        0
dtype: int64

## 2) daycare, park 데이터와 병합하기

In [ ]:
pip install geopy

In [ ]:
# # 도로명주소 위도 경도 값으로 바꿔주기
# from geopy.geocoders import Nominatim
# geo_local = Nominatim(user_agent='South Korea')

# # 위도, 경도 반환하는 함수
# def get_latitude_longitude(address):
#     location = geo_locator.geocode(address)
#     if location:
#          return location.latitude, location.longitude
#     else:
#          return None, None

In [ ]:
# train_df['latitude'], train_df['longitude'] = zip(*train_df['addr_kr'].apply(get_latitude_longitude))

# print(df)